<a href="https://colab.research.google.com/github/andreaaraldo/machine-learning-for-networks/blob/simplify-nn-case/04.neural_networks/04.neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you run this on Google Colab, change the environment to TPU or GPU

In [3]:
!pip install -U imblearn
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 604.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import datetime # Import the datetime module

from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import classification_report

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
tf.compat.v1.enable_eager_execution()
import random



!pip install category_encoders
import category_encoders as ce




# The following library is to plot the loss during training
# https://github.com/stared/livelossplot
! pip install livelossplot
from livelossplot import PlotLossesKerasTF


# Import the visualization library I prepared for you
! wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/visualization.py
from visualization import plot_conf_mat


# The following is to be able to mount Google Drive
from google.colab import drive

import os
from os.path import isfile

from functools import partial

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 6.6 MB/s eta 0:00:00
--2025-05-12 19:55:46--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/visualization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13023 (13K) [text/plain]
Saving to: ‘visualization.py’

visualization.py    100%[===================>]  12.72K  --.-KB/s    in 0s      

2025-05-12 19:55:46 (25.8 MB/s) - ‘visualization.

### Mount your Google Drive

Training a neural network may take long time and you don't want to do it every time. Once you've trained a model, it is better to save it, so that you can use it immediately for prediction next times.

Unfortunately, the storage on Google is reset every time. You need to store all persistent data in you Google Drive.

You need therefore to mount your Google Drive, which you will use later in this notebook

In [5]:
mount_point = '/content/gdrive' # Always the same, don't change it
drive.mount(mount_point, force_remount=True)
drive_path = mount_point + '/My Drive/' # Always the same, don't change it

# Replace the following folder with some folder inside your google drive
my_path = drive_path + \
  'tsp/teaching/data-science-for-networks/img-from-code/04.neural-networks/'

Mounted at /content/gdrive


# Use case description

The use case is from [ComCom19].


**Goal** Estimate bandwidth available between two communicating endpoints.
![alt text](https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/04.neural_networks/img/train-of-packets.png)

**Why**: Knowing the available bandwidth, video streaming clients can properly choose the quality level to request.



Measures
* Bits per seconds sent $r_{\text{in}}$
* Bits per second received $r_\text{out}$



![alt text](https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/04.neural_networks/img/r_in-r_out.png)
\[Figure from [ComCom19] \]


The auhtors set up the following testbed:


![alt text](https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/04.neural_networks/img/testbed.png)


Measures are collected in the **Video Receiver**. All the other machines just produce cross-traffic.

Measures are recorded via an Endace Data Acquisition and Generation (DAG) card, which timestamp all packets in an extremely precise way.

![alt text](https://www.endace.com/assets/images/products/DAG%209.5G4F_angled_small.png)

([Producer website](https://www.endace.com/endace-high-speed-packet-capture-solutions/oem/dag/))



For simplicity, we will just consider the case with a single link between client and server.

# Traces

The description of the dataset can be found in the Appendix of Khangura's [PhD thesis](https://www.repo.uni-hannover.de/bitstream/handle/123456789/9219/Khangura_Sukhpreet_PhD_Thesis.pdf?sequence=3&isAllowed=y). I have already pre-processed the dataset via [this script](https://github.com/andreaaraldo/machine-learning-for-networks/blob/master/04.neural_networks/ignore-preprocessing.ipynb), which you can ignore.

Let's directly download the preprocessed files and transform them into dataframes

In [6]:
!wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/refs/heads/simplify-nn-case/04.neural_networks/data/X_train.csv
!wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/refs/heads/simplify-nn-case/04.neural_networks/data/X_test.csv
!wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/refs/heads/simplify-nn-case/04.neural_networks/data/label_test.csv
!wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/refs/heads/simplify-nn-case/04.neural_networks/data/label_train.csv

X_train = pd.read_csv('X_train.csv', header=None)
X_test = pd.read_csv('X_test.csv', header=None)
label_test = pd.read_csv('label_test.csv', header=None, names=['label'], dtype=str)
label_train = pd.read_csv('label_train.csv', header=None, names=['label'], dtype=str)



--2025-05-12 19:56:12--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/refs/heads/simplify-nn-case/04.neural_networks/data/X_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105750 (103K) [text/plain]
Saving to: ‘X_train.csv’

X_train.csv         100%[===================>] 103.27K  --.-KB/s    in 0.02s   

2025-05-12 19:56:12 (4.14 MB/s) - ‘X_train.csv’ saved [105750/105750]

--2025-05-12 19:56:12--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/refs/heads/simplify-nn-case/04.neural_networks/data/X_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent

# Samples and features

In [7]:
X_train.head()

,0,1,2,3,4,5,6,7
0,4.417449,0.588993,0.294497,0.294497,0.294497,0.0,0.0,0.0
1,4.122952,0.588993,0.883490,0.294497,0.000000,0.0,0.0,0.0
2,3.239463,1.472483,0.883490,0.294497,0.000000,0.0,0.0,0.0
3,3.828456,0.588993,0.883490,0.588993,0.000000,0.0,0.0,0.0
4,4.122952,0.883490,0.588993,0.294497,0.000000,0.0,0.0,0.0


Each sample corresponds to an experiment, which is run for some minutes. A sample is described with a vector of 8 numbers, representing the distribution of $r=r_{in}/r_{out}$. These 8 numbers, per each experiment, are obtained as follows:

* First, in each experiment, several measures of $r$ are collected, i.e., at every few milliseconds
* The histogram of such measures is created, with 8 bars
* Each of the 8 values correspond to a bar

In [8]:
label_train.head()

,label
0,50
1,50
2,50
3,50
4,50


The $i$-th element of the vector `label_train` correspond to the bandwidth available during the $i$-th experiment

### Class imbalance

Let's check for class imbalance

In [9]:
label_train.value_counts()

,count
label,
25,400
50,400
12.5,100
37.5,100
75,100


Correct using Synthetic Minority Over-Sampling Technique (SMOTE) (see `03.classification.ipynb`)

In [10]:
smote = SMOTE()
X_train, label_train = smote.fit_resample(X_train, label_train)

label_train.value_counts()

,count
label,
12.5,400
25,400
37.5,400
50,400
75,400


### Scaling
With NN is important to **scale** the dataset

In [11]:
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print('X_train scaled\n', X_train[0:6, :] )

X_train scaled
 [[0.8        0.25       0.14285714 0.16666667 0.16666667 0.
  0.         0.        ]
 [0.73333333 0.25       0.42857143 0.16666667 0.         0.
  0.         0.        ]
 [0.53333333 0.625      0.42857143 0.16666667 0.         0.
  0.         0.        ]
 [0.66666667 0.25       0.42857143 0.33333333 0.         0.
  0.         0.        ]
 [0.73333333 0.375      0.28571429 0.16666667 0.         0.
  0.         0.        ]
 [0.6        0.5        0.42857143 0.16666667 0.         0.
  0.         0.        ]]


### One-hot encoding
**One-hot encode** the target

In [12]:
one_hot_encoder = ce.OneHotEncoder(cols=['label'], use_cat_names='True')
one_hot_encoder.fit(label_train)

OneHotEncoder(cols=['label'], use_cat_names='True')

In [13]:
one_hot_encoder = ce.OneHotEncoder(cols=['label'], use_cat_names='True')
one_hot_encoder.fit(label_train)
y_train = one_hot_encoder.transform(label_train)
y_test  = one_hot_encoder.transform(label_test)

print('y_train', y_train)

num_of_classes = y_train.shape[1]
class_names =list(y_train.columns)
print('There are ', num_of_classes, ' classes. Their names is:', class_names)

y_train       label_50  label_25  label_37.5  label_12.5  label_75
0            1         0           0           0         0
1            1         0           0           0         0
2            1         0           0           0         0
3            1         0           0           0         0
4            1         0           0           0         0
...        ...       ...         ...         ...       ...
1995         0         0           0           0         1
1996         0         0           0           0         1
1997         0         0           0           0         1
1998         0         0           0           0         1
1999         0         0           0           0         1

[2000 rows x 5 columns]
There are  5  classes. Their names is: ['label_50', 'label_25', 'label_37.5', 'label_12.5', 'label_75']


# Build a NN model

To train faster, change the runtime to GPU or TPU.
In the Google Colab menu at the top, do `Runtime > Change Runtime Type` and select GPU or TPU (sometimes GPU is faster than TPU, sometimes it is the opposite; so you can infidderently try either of them).

The following function creates a neural network architecture and compiles it

In [14]:
def make_sequential_model(sample_size, layer_sizes,
                          hidden_activation_function="relu",
                          out_activation_function="softmax",
                          loss_fun="categorical_crossentropy",
                          learning_rate=0.01,
                          regularization_coeff=0,
                          metrics=['accuracy']):
  """
  Makes a sequential model.
  Parameters
  -------------------------
  sample_size: integer
    The number of features of the samples

  layer_sizes: list
    List of the size of the neural network layers. For instance, if
    layer_sizes = [8, 6, 4], the 1st layer will have 8 neurons, the 2nd 6 etc.
    Attention: the size of the last layer (the output layer) is not arbitrary.
    In case of monodimensional regression, it must be 1.
    When using categorical_crossentropy, it must be the same as the number of
    categories.
    When using binary_crossentropy, it must be 1.

  inner_activation_function: string
    Activation function used in all layers, except the last one.
    Ex: "relu"

  out_activation_function: string
    Activation function of the last layer.
    Ex. "softmax"

  loss_fun: string
    The loss function we want to minimize. Ex. categorical_crossentropy

  learning_rate: float
    Ex. 0.01

  regularization_coeff: float
    Coefficient of ridge regression
    Ex. 0.01

  metrics: list of strings
    The metrics we want to show during training. Ex. ['accuracy']
  """

  model = Sequential()


  # In the next code we will use `partial`, which is a function of the ptyhon
  # library functools, which allows to define a class, identical to another
  # class but with some different default values.
  # In our case we define MyDenseLayer equal to the standard keras class
  # `Dense`, which implements a simple neural network layer, specifying
  # two default values: one for the activation function, and another for the
  # regularization

  if (regularization_coeff==0):
    # No regularization
    MyDenseLayer = partial(Dense, activation=hidden_activation_function)
  else:
    MyDenseLayer = partial(Dense, activation=hidden_activation_function,
                         kernel_regularizer=keras.regularizers.l2(regularization_coeff))

  # Add the input layer
  model.add( MyDenseLayer(layer_sizes[0],
                  input_dim = sample_size) )

  # Add hidden layers
  for i in range(1,len(layer_sizes)-1 ): # We iterate from the 2nd element to the penultimate
    model.add( MyDenseLayer(layer_sizes[i]) )

  # Add output layer
  model.add( Dense(layer_sizes[-1],
                  activation = out_activation_function) )


  model.compile(loss=loss_fun,
              optimizer=keras.optimizers.Adam(learning_rate=learning_rate) ,
              metrics=metrics)

  return model

See Ch.10 of [Ge19] to know more about these hyper-parameters.

Loss function:
* `categorical_crossentropy` is the cross-entropy as we defined in the slides
* In binary classification then use `sigmoid` activation function in the output layer instead of the `softmax` and `binary_crossentropy` loss.

Optimizer:
* We are using Stochastic Gradient Descent with learning rate $\eta=$ `learn_rate`

Metrics:
* We are asking Keras to show at each epoch the accuracy. Note that the accuracy value is ignored during training. This metric is just visualized

As en example, let us instantiate our first neural network

In [15]:
sample_size = X_train.shape[1] # num of columns
shallow_architecture = [8, 6, num_of_classes]
model = make_sequential_model(sample_size, shallow_architecture,
                              learning_rate=0.1)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │            54 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │            35 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 161 (644.00 B)

 Trainable params: 161 (644.00 B)

 Non-trainable params: 0 (0.00 B)

Let's introduce some operations to make our code **repeatable**: we want to have the same results, every time we run the different neural networks below, no matter the order in which we run them and no matter who runs them.

 To this aim, we use the following tricks:
 * `clear_session()`: it resets the neural network weights. If we do not do so, if we train a certain neural network A and then another neural network B, which may have the same architecture, some of the weights of B are initialized as in A. If we run in the opposite order, some of the weights of A are initialized as in B. Therefore, our results would depend on the order of training, which we do not want. This is why we will use `clear_session`. Actually, this function does even more: it resets all the variables handled by tensorflow. If you do not use it, every time you create a new neural network tensor flow will not remove the previous ones and adds the new one to its set of variables. This can bring to an excessive memory consumption. `clear_session` resets everything, thus solving memory problems.
 * Setting seeds: during training, there is some stochastic operations done: for instance, the validation set is chosen randomly, by default, the batch of samples is selected randomly at each iteration, etc.

More on randomness and reproducibility, on [this ODSC's blog](https://odsc.medium.com/properly-setting-the-random-seed-in-ml-experiments-not-as-simple-as-you-might-imagine-219969c84752).

In [16]:
def enforce_reproducibility(seed):
  tf.keras.backend.clear_session()

  # To know more:
  #       https://machinelearningmastery.com/reproducible-results-neural-networks-keras/
  random.seed(seed)
  np.random.seed(random.randint(0,300000))
  tf.random.set_seed(random.randint(0,300000))

This network is probably too big with respect to the small amount of data. However, nobody can tell us, until we see the training results.

**Always experiment with NNs of different sizes**

# Training

In [17]:
def train_model(model, nn_file, X_tr, y_tr, seed, max_epochs=1000,
                overwrite=True, validation_split=0.2, patience=20):
  """
  model: neural network model
            It must be a compiled neural network, e.g., a model issued by the
            function make_sequential_model(..) defined before

  nn_file:  string (name of a file)
            This file will be used to store the weights of the trained neural
            network. Such weights are automatically stored during training
            (thanks to the ModelCheckpoint callback (see the implementation
            code)), so that even if the code fails in the middle of training,
            you can resume training without starting from scratch.
            If the file already exists, before starting training, the weights
            in such a file will be loaded, so that we do not start training from
            scratch, but we start already from (hopefully) good weigths.

  overwrite: boolean
            If true, the model will be built and trained from scratch,
            indipendent of whether nn_file exists or not.

  seed: integer

  X_tr: matrix
            Feature matrix of the training set

  y_tr: matrix
            True labels of the training set

  max_epochs: integer
            Training will stop after such number of epochs

  validation_split: float (between 0 and 1)
            Fraction of training dataset that will be used as validation

  patience: integer
            Training will stop if the validation loss does not improve after the
            specified number of epochs
  """

  enforce_reproducibility(seed)


  # Before starting training, Keras divides (X_tr, y_tr) into a training subset
  # and a validation subset. During iterations, Keras will do backpropagation
  # in order to minimize the loss on the trainins subset, but it will monitor
  # and also plot the loss on the validation subset.
  # However, Keras always takes the first part of (X_tr, y_tr) as training
  # subset and the second part as validation subset. This can be bad, in case
  # the dataset has been created with a certain order (for instance all the
  # samples with a certain characteristic first, and then all the others), as
  # we instead need to train the neural network on a representative subset of
  # samples. For this reason, we first shuffle the dataset
  X_train, y_train = shuffle(X_tr, y_tr, random_state=seed)


  ##################
  #### CALLBACKS ###
  ##################
  # These functions are called at every epoch
  log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  plot_cb = PlotLossesKerasTF()  # Plots the loss # new code should be tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1) #
  checkpoint_cb = ModelCheckpoint(nn_file) # Stores weights
  logger_cb = CSVLogger(nn_file+'.csv', append=True) # Stores history
                # see https://theailearner.com/2019/07/23/keras-callbacks-csvlogger/


  # To stop early if we already converged
  # See pagg 315-16 of [Ge19]
  early_stop_cb = tf.keras.callbacks.EarlyStopping(verbose=1,
      monitor='val_loss',
     patience=patience, restore_best_weights=True)


  if overwrite==True:
    try:
      os.remove(nn_file)
    except OSError:
      pass

    try:
      os.remove(nn_file+'.csv')
    except OSError:
      pass

  if isfile(nn_file):
    print("Loading pre-existing model")
    model = load_model(nn_file)

  history = model.fit(X_train, y_train, epochs=max_epochs,
                      callbacks = [plot_cb, checkpoint_cb, logger_cb, early_stop_cb],
                      validation_split=validation_split )

  return history

In [18]:
nn_file = my_path + 'nn-0.1.h5'

history = train_model(model, nn_file, X_train, y_train, seed=5, max_epochs=200)

Epoch 1/200
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6085 - loss: 0.9735

50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6132 - loss: 0.9612 - val_accuracy: 0.7775 - val_loss: 0.4413
Epoch 2/200
26/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8351 - loss: 0.4021 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8319 - loss: 0.3966 - val_accuracy: 0.8100 - val_loss: 0.4251
Epoch 3/200
26/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8603 - loss: 0.3615 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8541 - loss: 0.3604 - val_accuracy: 0.7925 - val_loss: 0.4298
Epoch 4/200
28/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8443 - loss: 0.3661 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8416 - loss: 0.3625 - val_accuracy: 0.8000 - val_loss: 0.4503
Epoch 5/200
28/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8462 - loss: 0.3655 

...

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8519 - loss: 0.3630 - val_accuracy: 0.8200 - val_loss: 0.4592
Epoch 25/200
30/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8733 - loss: 0.3537 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8682 - loss: 0.3455 - val_accuracy: 0.8250 - val_loss: 0.4331
Epoch 26/200
29/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8613 - loss: 0.3471 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8475 - loss: 0.3572 - val_accuracy: 0.8225 - val_loss: 0.4102
Epoch 27/200
28/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8317 - loss: 0.3532 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8353 - loss: 0.3470 - val_accuracy: 0.8275 - val_loss: 0.4256
Epoch 28/200
28/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8521 - loss: 0.3530 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8514 - loss: 0.3464 - val_accuracy: 0.8100 - val_loss: 0.4032
Epoch 28: early stopping
Restoring model weights from the end of the best epoch: 8.


Note that training ended before the 200 epochs specified as `max_epochs`. This is because we set the value of `patience` to 20: since for 20 consecutive epochs the validation loss was not improving, we aborted training, to avoid wasting time.

In any case, our NN does has not managed to converge. This is often due to the **learning rate being too large**. In our case, it was 0.1. Let's try a smaller one.

In [19]:
model = make_sequential_model(sample_size, shallow_architecture,
                              learning_rate=0.001)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
nn_file = my_path + 'nn-0.001.h5'

history = train_model(model, nn_file, X_train, y_train, seed=5, max_epochs=200)

Epoch 1/200
26/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3108 - loss: 1.6227   

50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3352 - loss: 1.6092 - val_accuracy: 0.4225 - val_loss: 1.5529
Epoch 2/200
27/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3400 - loss: 1.5697 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3779 - loss: 1.5566 - val_accuracy: 0.4825 - val_loss: 1.4998
Epoch 3/200
29/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4556 - loss: 1.5080 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4805 - loss: 1.4913 - val_accuracy: 0.5125 - val_loss: 1.4068
Epoch 4/200
26/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5355 - loss: 1.3964 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5570 - loss: 1.3664 - val_accuracy: 0.5025 - val_loss: 1.2713
Epoch 5/200
28/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5676 - loss: 1.2432 

...

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8698 - loss: 0.3041 - val_accuracy: 0.8250 - val_loss: 0.3611
Epoch 167/200
27/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8649 - loss: 0.3158 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8698 - loss: 0.3039 - val_accuracy: 0.8250 - val_loss: 0.3609
Epoch 168/200
29/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8657 - loss: 0.3138 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8698 - loss: 0.3038 - val_accuracy: 0.8250 - val_loss: 0.3611
Epoch 169/200
30/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8660 - loss: 0.3127 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8698 - loss: 0.3036 - val_accuracy: 0.8250 - val_loss: 0.3611
Epoch 170/200
29/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8657 - loss: 0.3134 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8698 - loss: 0.3034 - val_accuracy: 0.8250 - val_loss: 0.3612
Epoch 170: early stopping
Restoring model weights from the end of the best epoch: 150.


Much better now. Let's try with a much smaller one.

In [21]:
nn_file = my_path + 'nn-0.00001.h5'
model = make_sequential_model(sample_size, shallow_architecture,
                              learning_rate=0.00001)
history = train_model(model, nn_file, X_train, y_train, seed=5, max_epochs=200)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
26/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3038 - loss: 1.6297   

50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3204 - loss: 1.6228 - val_accuracy: 0.3875 - val_loss: 1.6011
Epoch 2/200
27/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3075 - loss: 1.6287 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3234 - loss: 1.6222 - val_accuracy: 0.3875 - val_loss: 1.6005
Epoch 3/200
28/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3098 - loss: 1.6276 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3249 - loss: 1.6216 - val_accuracy: 0.3875 - val_loss: 1.6000
Epoch 4/200
29/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3112 - loss: 1.6265 

...

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4447 - loss: 1.5159 - val_accuracy: 0.4775 - val_loss: 1.5025
Epoch 197/200
30/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4285 - loss: 1.5215 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4445 - loss: 1.5153 - val_accuracy: 0.4800 - val_loss: 1.5020
Epoch 198/200
29/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4273 - loss: 1.5215 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4443 - loss: 1.5146 - val_accuracy: 0.4800 - val_loss: 1.5014
Epoch 199/200
28/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4245 - loss: 1.5215 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4430 - loss: 1.5140 - val_accuracy: 0.4775 - val_loss: 1.5008
Epoch 200/200
29/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4275 - loss: 1.5202 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4445 - loss: 1.5133 - val_accuracy: 0.4775 - val_loss: 1.5002
Restoring model weights from the end of the best epoch: 200.


The loss decreases too slowly
. Let's try a learning rate in the middle

In [22]:
nn_file = my_path + 'nn-0.0001.h5'
model = make_sequential_model(sample_size, shallow_architecture,
                              learning_rate=0.0001)
history = train_model(model, nn_file, X_train, y_train, seed=5, max_epochs=200)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
48/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3229 - loss: 1.6221

50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3245 - loss: 1.6215 - val_accuracy: 0.3950 - val_loss: 1.5963
Epoch 2/200
49/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3330 - loss: 1.6160

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3340 - loss: 1.6156 - val_accuracy: 0.4075 - val_loss: 1.5911
Epoch 3/200
27/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3243 - loss: 1.6171 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3410 - loss: 1.6100 - val_accuracy: 0.4125 - val_loss: 1.5861
Epoch 4/200
27/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3268 - loss: 1.6117 

...

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8358 - loss: 0.6075 - val_accuracy: 0.7900 - val_loss: 0.6734
Epoch 197/200
29/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8364 - loss: 0.6075 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8368 - loss: 0.6053 - val_accuracy: 0.7900 - val_loss: 0.6714
Epoch 198/200
30/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8376 - loss: 0.6051 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8379 - loss: 0.6032 - val_accuracy: 0.7900 - val_loss: 0.6693
Epoch 199/200
29/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8392 - loss: 0.6031 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8392 - loss: 0.6011 - val_accuracy: 0.7900 - val_loss: 0.6674
Epoch 200/200
29/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8392 - loss: 0.6010 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8392 - loss: 0.5989 - val_accuracy: 0.7900 - val_loss: 0.6653
Restoring model weights from the end of the best epoch: 200.


Let's try with learning rate 0.01

In [23]:
nn_file = my_path + 'nn-0.01.h5'
model = make_sequential_model(sample_size, shallow_architecture,
                              learning_rate=0.01)
history = train_model(model, nn_file, X_train, y_train, seed=5, max_epochs=200)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4163 - loss: 1.4835

50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4183 - loss: 1.4797 - val_accuracy: 0.5050 - val_loss: 0.9757
Epoch 2/200
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7128 - loss: 0.7952

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7140 - loss: 0.7930 - val_accuracy: 0.7825 - val_loss: 0.5948
Epoch 3/200
28/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8438 - loss: 0.4761 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8393 - loss: 0.4648 - val_accuracy: 0.8050 - val_loss: 0.4798
Epoch 4/200
27/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8592 - loss: 0.3764 

...

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8689 - loss: 0.3044 - val_accuracy: 0.8225 - val_loss: 0.3619
Epoch 69/200
27/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8717 - loss: 0.3142 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8715 - loss: 0.3047 - val_accuracy: 0.8275 - val_loss: 0.3625
Epoch 70/200
28/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8676 - loss: 0.3124 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8682 - loss: 0.3034 - val_accuracy: 0.8225 - val_loss: 0.3634
Epoch 71/200
27/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8713 - loss: 0.3154 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8711 - loss: 0.3047 - val_accuracy: 0.8225 - val_loss: 0.3635
Epoch 72/200
28/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8661 - loss: 0.3139 

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8681 - loss: 0.3041 - val_accuracy: 0.8225 - val_loss: 0.3615
Epoch 72: early stopping
Restoring model weights from the end of the best epoch: 52.


## Recover the best model

Observing the validation loss curves, it seems that learning rate 0.01 is the best.

Let us thus recover the model with 0.001. Remember that our code saves automatically all the models. Therefore, we can load the already trained model from the respective file.

We also do some additional 300 epochs of training. Note that we do not start training from scratch, since the initial NN weights are the ones that we had at the end of the previous training, just before saving the model in the file. You have thus understood that saving models in files is a good practice, because we can also improve such models over and over again, even several days after our last training.

To load a previously existing model, we specify `overwrite=False`.

In [24]:
nn_file = my_path + 'nn-0.01.h5'
model = make_sequential_model(sample_size, shallow_architecture,
                              learning_rate=0.01, )
history = train_model(model, nn_file, X_train, y_train, seed=5, max_epochs=300,
                      overwrite=False)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Loading pre-existing model
Epoch 1/300


NotImplementedError: numpy() is only available when eager execution is enabled.

Don't be mislead by the zoom! The model seems visually more "instable" than when we have saved it, but it is not. If you look at the y-axis, you can notice they are zoomed in a very small interval. So, oscillations seem big, but they are actually very small.

# Prediction

Let's use the model for prediction.

In [ ]:
nn_file = my_path + 'nn-0.01.h5'
model = load_model(nn_file)

Get the probability for each sample of being in a class.

In [ ]:
y_proba = model.predict(X_test)
print( 'y_proba\n', y_proba)

The predicted class of each sample is the one with the highest probability.

In [ ]:
pred_proba = model.predict(X_test)
pred_label = np.argmax(pred_proba,axis=1)
print( 'pred_label\n', pred_label )

The best way to evaluate the model is the confusion matrix

In [ ]:
y_test.values

We need to convert one-hot-encoded values to numerical labels

In [ ]:
true_label = np.argmax(y_test.values, axis=1)
true_label

In [ ]:
plot_conf_mat(true_label, pred_label, np.array(class_names) )

In [ ]:
print("The names of the classes are ", class_names)
print("The true classes found in the test set are ", Counter(true_label) )
print("The predicted classes are ", Counter(pred_label) )

Observe that class `3`, i.e., `label_37.5` is predicted by the model but does not appear in the test set. This is why it appear as `nan` in the confusion matrix.

Observe also that class `4`, i.e., `label_12.5` does not appear neither in the true labels of the test set, nor in the predicted labels. For this reason it does not appear in the confusion matrix.

In [ ]:
print( classification_report(true_label, pred_label, labels=[0,1,2,3,4],
                             target_names=class_names) )

# Deeper architecture

Let us now try a deeper neural network and check if it is better.

In [ ]:
deep_architecture = [8, 8 ,8, 8, 8, 8, 8, 8, 8, num_of_classes]

In [ ]:
model = make_sequential_model(sample_size, deep_architecture,
                              learning_rate=0.01)
model.summary()

In [ ]:
nn_file = my_path + 'nn-deep-lr_0.01.h5'
history = train_model(model, nn_file, X_train, y_train, seed=5, max_epochs=300)

It has not converged. With such a deep neural network, we might need a **smaller learning rate**.

In [ ]:
nn_file = my_path + 'nn-deep-lr_0.001.h5'
model = make_sequential_model(sample_size, deep_architecture,
                              learning_rate=0.001)
history = train_model(model, nn_file, X_train, y_train, seed=5, max_epochs=200)

Or even smaller

In [ ]:
nn_file = my_path + 'nn-deep-lr_0.0001.h5'
model = make_sequential_model(sample_size, deep_architecture,
                              learning_rate=0.0001)
history = train_model(model, nn_file, X_train, y_train, seed=5, max_epochs=200)

The validation loss was still decreasing. Maybe if we continue training, we can further improve the validation loss. Let's try to resume the training where we have left it, by using `overwrite=False`

In [ ]:
history = train_model(model, nn_file, X_train, y_train, seed=5, max_epochs=300,
                      overwrite=False)

Let's furthur continue

In [ ]:
history = train_model(model, nn_file, X_train, y_train, seed=5, max_epochs=300,
                      overwrite=False)

After 800 epochs of training for the `deep_architecture`, the validation loss remain higher than with a smaller neural network. We may further improve `deep_architecture by keep searching for a better learning rate, using regularization, etc. All these things are worth trying, but take time.

This shows that, although a deeper neural network is more powerfull than a shallower one, in practice it may difficult and time-consuming to train. In that case, using a shallower neural network is better.

Suggestion: do not start directly with deep and big networks. Start shallow and narrow, and increase after (and gradually).

# Further improvements

We still need to improve. You can play with hyper-parameters (see slides):
* Try other architectures (more shallow, even deeper, larger (each layer having a lot of neurons), narrower (each layer having few neurons), etc.
* Regularize (use the parameter `regularization_coeff` of the function `make_sequential` defined in this noteboook)
* Change the optimizer
* Change the activation functions
* Change the batch size (default in `model.fit(..)` is 32)

# Limits of the work

* Only a finite set of available bandwidth values are used (12.5, 25, 37.5, 50, 75 Mbps). In reality, any value can occur => Need to extend the test and validation test with random bandwidth values.
* Is the method precise also when the total capacity is not 100 Mbps?

# Regression
We have so far only used neural network to perform classification. It is also possible to perform regression. To do so, the code remains essentially the same, just change the last layer: instead of `softmax`, put `Dense(1)` (see pagg. 307-308 of [Ge19]).



# References

* [ComCom19] Khangura, S. K., Fidler, M., & Rosenhahn, B. (2019). Machine learning for measurement-based bandwidth estimation. Computer Communications, 144(1), 18–30.
* [Ge19] Geron, Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2019, O'Reilly
* [KhConf19] Khangura, S. K. (2019). Neural Network-based Available Bandwidth Estimation from TCP Sender-side Measurements. In IEEE/IFIP PEMWN.
* [KhThesis19] Khangura, S. K. (2019). Machine Learning-based Available Bandwidth Estimation. Leibniz University.